In [4]:
import pandas as pd
import numpy as np
import json
import requests

In [5]:
data = requests.get("http://github.com/hodcroftlab/covariants/raw/master/cluster_tables/USAClusters_data.json").json()

# with open("../data/variants.json", "w") as file:
#     json.dump(data, file, indent=2)

In [36]:
def parse_variants_json(data):
    df_list = []
    for state, data in data["countries"].items():
        df_state = pd.DataFrame.from_dict(data) 

        df_state = df_state.melt(
            id_vars="week", 
            value_vars=[ c for c in df_state.columns if c != "week" and c != "total_sequences" ],
            var_name="variant",
            value_name="cases"
        ).rename(columns={"week": "date"})
        
        df_state.insert(0, "state", state)

        df_list.append(df_state)

    return pd.concat(df_list)

In [37]:
df_full = parse_variants_json(data)
print(df_full.shape)
df_full.head()

(69906, 4)


,state,date,variant,cases
0,California,2020-04-27,"20I (Alpha, V1)",0.0
1,California,2020-05-11,"20I (Alpha, V1)",0.0
2,California,2020-05-25,"20I (Alpha, V1)",0.0
3,California,2020-06-08,"20I (Alpha, V1)",0.0
4,California,2020-06-22,"20I (Alpha, V1)",0.0


In [39]:
df_full.to_parquet("../data/variants.parquet", engine="pyarrow", compression="brotli")